# Encryption Protocol Performance

## Install Dependencies

In [ ]:
!apt update

# Install OpenSSL
!apt install -y openssl

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [976 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Ge

In [ ]:
!openssl version

OpenSSL 3.0.2 15 Mar 2022 (Library: OpenSSL 3.0.2 15 Mar 2022)


## Performance Test of OpenSSL's Algorithms

In [22]:
# List all SHA1 algorithm
!openssl list -cipher-algorithms | grep rc
!openssl list -cipher-algorithms | grep sha1
!openssl list -cipher-algorithms | grep bf
!openssl list -cipher-algorithms | grep dsa

  rc2 => RC2-CBC
  rc2-128 => RC2-CBC
  rc2-40 => RC2-40-CBC
  rc2-64 => RC2-64-CBC
  bf => BF-CBC


## SHA1

In [33]:
# Read first few characters of plaintext file
!head -c 100 plaintext1.txt

Gykyfjb14EhdCvaFU0wx3KYuynfc7MbpPUqnF5cf17ACDFbx0P0cRK1H6H9fRhWFXTJiGbE1QKa57QeU13JKHY1ZaHWWqW4WfS1D

In [15]:
# Get file size
!ls -lh *.txt

-rw-r--r-- 1 root root 977K Sep 10 07:03 plaintext1.txt


In [16]:
# Measure speed of sha1 using OpenSSL encryption speed
!time openssl dgst -sha1 plaintext1.txt

SHA1(plaintext1.txt)= 93e0301be1cda84eb9143a1d50ede885992230fd

real	0m0.020s
user	0m0.005s
sys	0m0.005s


## RC4

In [25]:
# Measure speed of rc4 using OpenSSL encryption speed
!time openssl enc -rc4 -in plaintext1.txt -out ciphertext.bin -K "00112233445566778899AABBCCDDEEFF" -nosalt -nopad -provider legacy


real	0m0.013s
user	0m0.008s
sys	0m0.005s


In [26]:
# Measure speed of rc4 using OpenSSL decryption speed
!time openssl enc -rc4 -d -in ciphertext.bin -out decryptedtext1.txt -K "00112233445566778899AABBCCDDEEFF" -nosalt -nopad -provider legacy


real	0m0.014s
user	0m0.007s
sys	0m0.006s


## Blowfish

In [27]:
# Measure speed of rc4 using OpenSSL encryption speed
!time openssl enc -bf-cbc -in plaintext1.txt -out ciphertext.bin -K "00112233445566778899AABBCCDDEEFF" -iv "0102030405060708" -nosalt -nopad -provider legacy


real	0m0.020s
user	0m0.016s
sys	0m0.004s


In [28]:
# Measure speed of rc4 using OpenSSL decryption speed
!time openssl enc -bf-cbc -d -in ciphertext.bin -out decryptedtext1.txt -K "00112233445566778899AABBCCDDEEFF" -iv "0102030405060708" -nosalt -nopad -provider legacy


real	0m0.023s
user	0m0.012s
sys	0m0.010s


## DSA

In [34]:
# Generate DSA Param
!time openssl dsaparam -out dsaparam.pem 2048


real	0m1.312s
user	0m1.234s
sys	0m0.006s


In [45]:
# Generate Private Key
!time openssl gendsa -out privatekey.pem dsaparam.pem


real	0m0.012s
user	0m0.009s
sys	0m0.002s


In [46]:
# Extract Public Key
!time openssl dsa -in privatekey.pem -pubout -out publickey.pem

read DSA key
writing DSA key

real	0m0.018s
user	0m0.010s
sys	0m0.002s


In [47]:
# Sign file
!time openssl dgst -sha256 -sign privatekey.pem -out signature.bin plaintext1.txt


real	0m0.016s
user	0m0.008s
sys	0m0.006s


In [48]:
# Verifying signature
!time openssl dgst -sha256 -verify publickey.pem -signature signature.bin plaintext1.txt

Verified OK

real	0m0.014s
user	0m0.008s
sys	0m0.004s
